In [1]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch
torch.use_deterministic_algorithms(True)
import numpy as np
import pandas as pd
import random
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import tensorflow as tf

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)



from model.Vec2Image_py.get_matrix import get_matrix
from model.Vec2Image_py.trainer import get_net_trainer
from model.Vec2Image_py.getNETtest import NetTester
from model.Vec2Image_py.getSMOTE import SmoteGenerator
from model.Vec2Image_py.getPrioritizeGene import getPrioritizeGene
from model.Vec2Image_py.Cart2Pixel import Cart2Pixel
from model.Vec2Image_py.ConvPixel import ConvPixel
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder


# 1. Vec2Image_data_processing

In [2]:

def Vec2Image_data_processing(filepath, Parm):
    print("\nLoading full gene expression dataset...")
    df = pd.read_csv(filepath)
    set_seed(42)

    X_full = df.iloc[:, 1:].to_numpy(dtype=np.float32)
    labels_full = [col.split('.')[0] for col in df.columns[1:]]
    label_map = {label: idx for idx, label in enumerate(sorted(set(labels_full)))}
    y_full = np.array([label_map[l] for l in labels_full])
    num_classes = len(label_map)

    print(f"Number of classes: {num_classes}")
    print("\nLabel to index mapping:")
    for label, idx in label_map.items():
        print(f"  {label}: {idx}")


    dset = {
        'Xtrain': X_full,
        'train_labels': y_full.astype(int),
    }
    print("\nRunning get_matrix...")
    Out = get_matrix(dset, Parm)

    print("\nApplying SMOTE...")
    smote = SmoteGenerator(dset['Xtrain'].T, dset['train_labels'], seed=42)
    X_aug, y_aug = smote.fit_resample()
    X_aug = X_aug.T

    print("\nTesting ConvPixel...")
    sample_vec = dset['Xtrain'][:, 0]
    image = ConvPixel(sample_vec, dset['xp'], dset['yp'], dset['A'], dset['B'], dset['Base'])
    print("Image shape:", image.shape)

    print("\nEncoding labels...")
    le = LabelEncoder()
    y_aug_int = le.fit_transform(y_aug)
    y_val_int = le.transform(dset['Validation_labels'])

    y_train_oh = np.eye(len(le.classes_))[y_aug_int]
    y_val_oh = np.eye(len(le.classes_))[y_val_int]
    dset['label_encoder'] = le


    print("\nConverting augmented data to images...")
    X_train_imgs = np.zeros((X_aug.shape[1], 1, dset['A'], dset['B']), dtype=np.float32)
    for i in range(X_aug.shape[1]):
        fvec = X_aug[:, i][Out['feature_order']] 
        X_train_imgs[i, 0, :, :] = ConvPixel(fvec, dset['xp'], dset['yp'], dset['A'], dset['B'], dset['Base'], 0)


    print("\nPreparing validation images...")
    X_val_imgs = dset['XValidation'].transpose(3, 2, 0, 1)

    print("\nFinished dataset preparation.")
    return {
        'X_train_imgs': X_train_imgs,
        'X_val_imgs': X_val_imgs,
        'y_train_oh': y_train_oh,
        'y_val_oh': y_val_oh,
        'label_map': label_map,
        'num_classes': num_classes,
        'original_dataset': dset,
        'Out': Out,
        'dset': dset
    }



In [3]:
Parm = {
    'Method': 'tSNE',
    'Max_Px_Size': 30,
    'MPS_Fix': 1,
    'ValidRatio': 0.2,
    'Seed': 42,
    'NORM': 1
}

In [4]:
filepath="data/Darmanis.csv"
Process_Vec2Image = Vec2Image_data_processing(filepath=filepath, Parm=Parm)


Loading full gene expression dataset...
Number of classes: 9

Label to index mapping:
  OPC: 0
  astrocytes: 1
  endothelial: 2
  fetal_quiescent: 3
  fetal_replicating: 4
  hybrid: 5
  microglia: 6
  neurons: 7
  oligodendrocytes: 8

Running get_matrix...

NORM-1
Selecting top 900 genes by variance...
tSNE with exact algorithm is used

 Pixels: 31 x 31

Applying SMOTE...

Testing ConvPixel...
Image shape: (31, 31)

Encoding labels...

Converting augmented data to images...

Preparing validation images...

Finished dataset preparation.


# 2. Vec2Image_model_train

In [5]:
def Vec2Image_model_train(Process_Vec2Image):
    set_seed(42)
    print("\nTraining model...")
    model = get_net_trainer(
        Process_Vec2Image['X_train_imgs'],
        Process_Vec2Image['y_train_oh'],
        Process_Vec2Image['X_val_imgs'],
        Process_Vec2Image['y_val_oh'],
      
    )

    Process_Vec2Image['Out'].update({'model': {'net': model}})


In [6]:
Vec2Image_model_train(Process_Vec2Image)


Training model...
Epoch 01: Loss=61.0403 | Train Acc=28.10% | Val Acc=17.58%
Epoch 02: Loss=50.0994 | Train Acc=59.69% | Val Acc=30.77%
Epoch 03: Loss=36.5142 | Train Acc=81.48% | Val Acc=63.74%
Epoch 04: Loss=22.2392 | Train Acc=89.32% | Val Acc=78.02%
Epoch 05: Loss=12.8240 | Train Acc=96.19% | Val Acc=81.32%
Epoch 06: Loss=7.5769 | Train Acc=98.47% | Val Acc=82.42%
Epoch 07: Loss=4.9691 | Train Acc=99.24% | Val Acc=83.52%
Epoch 08: Loss=3.1429 | Train Acc=99.78% | Val Acc=83.52%
Epoch 09: Loss=2.2888 | Train Acc=99.89% | Val Acc=82.42%
Epoch 10: Loss=1.5738 | Train Acc=99.89% | Val Acc=84.62%
Epoch 11: Loss=1.3090 | Train Acc=100.00% | Val Acc=83.52%
Epoch 12: Loss=1.0344 | Train Acc=100.00% | Val Acc=82.42%
Epoch 13: Loss=0.8046 | Train Acc=100.00% | Val Acc=80.22%
Epoch 14: Loss=0.6982 | Train Acc=100.00% | Val Acc=80.22%
Epoch 15: Loss=0.5830 | Train Acc=100.00% | Val Acc=82.42%
Epoch 16: Loss=0.5872 | Train Acc=100.00% | Val Acc=81.32%
Epoch 17: Loss=0.4709 | Train Acc=100.00% 

# 3. Vec2Image_model_test

In [7]:
def Vec2Image_model_test(Out, dset, device='cpu'):
    print("\n===== Running model test =====")
    set_seed(42)
    # 提取并处理验证集原始数据
    X_val_raw = np.nan_to_num(Out['ValidationRawdata'])
    gene_idx = Out['selected_gene_idx']
    X_val_raw = X_val_raw[gene_idx, :]

    dset['Xtest'] = X_val_raw
    dset['test_labels'] = Out['ValidationLabelsOrdered']
    n_val = dset['Xtest'].shape[1]
    dset['test_labels'] = dset['test_labels'][:n_val]

    # 执行测试
    tester = NetTester(dset, Out, device=device)
    acc, XTest_tensor, Y_pred = tester.run_test()

    # 输出信息
    print(f"\nTest accuracy: {acc:.2%}")
    print("XP hash:", hash(tuple(dset['xp'])))
    print("YP hash:", hash(tuple(dset['yp'])))
    print("Test Out XP hash:", hash(tuple(Out['xp'])))
    print("First few test labels:", dset['test_labels'][:5])
    print("Xtest shape:", dset['Xtest'].shape)



In [8]:
if isinstance(Process_Vec2Image['Out']['model']['net'], tuple):
    Process_Vec2Image['Out']['model']['net'] = Process_Vec2Image['Out']['model']['net'][0]
Vec2Image_model_test(Process_Vec2Image['Out'], Process_Vec2Image['dset'], device='cpu')


===== Running model test =====

Using Norm-1 ...

Test accuracy: 83.52%
XP hash: 8749714411047054541
YP hash: -4047352006026151336
Test Out XP hash: 8749714411047054541
First few test labels: [0 0 0 1 1]
Xtest shape: (900, 91)


# 4. Vec2Image_features_importance

In [9]:
def Vec2Image_features_importance(dest, Out, k, num_classes, device='cpu'):
    set_seed(42)
    if Out['Norm'] == 1:
        print('\nNORM-1\n')
        Out['Max'] = np.max(dest['Xtest'], axis=1, keepdims=True)
        Out['Min'] = np.min(dest['Xtest'], axis=1, keepdims=True)
        dest['Xtest'] = (dest['Xtest'] - Out['Min']) / (Out['Max'] - Out['Min'])
        dest['Xtest'][np.isnan(dest['Xtest'])] = 0

    elif Out['Norm'] == 2:
        print('\nNORM-2\n')
        Out['Min'] = np.min(dest['Xtest'], axis=1, keepdims=True)
        dest['Xtest'] = np.log(dest['Xtest'] + np.abs(Out['Min']) + 1)
        Out['Max'] = np.max(dest['Xtest'])
        dest['Xtest'] = dest['Xtest'] / Out['Max']

    countgene = min(dest['Xtest'].shape[0], len(Out['xp']), len(Out['yp']))
    error = np.zeros(countgene)

    print(f"\nCalculating feature importance for {countgene} genes...")
    
    for i in range(countgene):
        shuffledata = np.copy(dest['Xtest'])
        neigh = NearestNeighbors(n_neighbors=k, p=5, metric='minkowski')
        neigh.fit(np.column_stack((Out['xp'], Out['yp'])))
        mIdx = neigh.kneighbors([[Out['xp'][i], Out['yp'][i]]], return_distance=False)[0]
        mIdx = mIdx[mIdx < shuffledata.shape[0]]  
        shuffledata[mIdx, :] = 1

        num_test_labels = len(dest['test_labels'])
        sample_pixel = ConvPixel(shuffledata[:, 0], Out['xp'], Out['yp'], Out['A'], Out['B'], Out['Base'], 0)
        height, width = sample_pixel.shape
        M = np.zeros((height, width, 1, num_test_labels))

        M[:, :, 0, 0] = sample_pixel

        for j in range(1, num_test_labels):
            M[:, :, 0, j] = ConvPixel(shuffledata[:, j], Out['xp'], Out['yp'], Out['A'], Out['B'], Out['Base'], 0)

        # print(M.shape)
        X_test_tensor = torch.from_numpy(M).permute(3, 2, 0, 1).float().to(device)

        Y_pred = Out['model']['net'](X_test_tensor)
        Y_pred = torch.argmax(Y_pred, dim=1).cpu().numpy()

        valError = np.mean(Y_pred == dest['test_labels'])


        error[i] = valError
        # print(f'the running gene number is {i}')

    GeneRank = error

    top_k_indices = np.argsort(GeneRank)[:num_classes]

    top_gene_idx = Out['selected_gene_idx']         
    true_gene_idx = top_gene_idx[top_k_indices]
    print(f"Top {num_classes} important genes(original):", true_gene_idx)
    return true_gene_idx

In [10]:
true_gene_idx = Vec2Image_features_importance(
    Process_Vec2Image['dset'], 
    Process_Vec2Image['Out'], 
    k=5, 
    num_classes=Process_Vec2Image['num_classes'], 
    device='cpu'
)

# 存到txt文件
output_file = "Vec2Image_top_genes.txt"
with open(output_file, 'w') as f:
    for gene in true_gene_idx:
        f.write(f"{gene}\n")


NORM-1


Calculating feature importance for 375 genes...
Top 9 important genes(original): [18473 19684  5862  3088 17587  7832  2353 13246 16680]
